In [1]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
def process_data(file):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    return np.array(coord)

In [160]:
def read_directory(directory, sample_rate):
    point_array_all = []
    land_mark_array_all=[]

    for filename in sorted(os.listdir(directory)):

        if filename.endswith(".pts"): 
            path=os.path.join(directory, filename)
            #print("Processing:",path)
            points = open(path)
            point_array = process_data(points)

            point_array_sampled = []
            
            #sampling
            for i in range(0,len(point_array),sample_rate):
                point_array_sampled.append(point_array[i])

            point_array_sampled = np.array(point_array_sampled)

            point_array_all.append(point_array_sampled)

            continue
        elif filename.endswith(".lan"):
            path = os.path.join(directory, filename)
            #print("Processing:",path)
            landmarks = open(path)
            landmark_array = process_data(landmarks)

            land_mark_array_all.append(landmark_array)

            continue
            
    return point_array_all,land_mark_array_all

In [108]:
def calculate_transformation_matrix(original, aligned):
    transform_matrices=[]
    for i in range(0, len(original)):
        transform_matrix = np.zeros(shape=(3,4))
        
        transform_matrix[0][-1] = aligned[i][0][0] - original[i][0][0]
        transform_matrix[0][0] = 1
        
        transform_matrix[1][-1] = aligned[i][0][1] - original[i][0][1]
        transform_matrix[1][1] = 1
        
        transform_matrix[2][-1] = aligned[i][0][2] - original[i][0][2]
        transform_matrix[2][2] = 1
        
        transform_matrices.append(transform_matrix)
    
    return transform_matrices

In [139]:
def calculate_aligned_landmarks(landmarks, transform_matrices):
    updated_landmarks_all = []
    
    for i in range(0, len(landmarks)):
        updated_landmarks=[]
        
        if(len(landmarks[i])!=0):
            landmarks_1 = np.c_[landmarks[i],np.ones(len(landmarks[i]))]
                
            for j in range(0, len(landmarks[i])):
                
                updated_landmark=[]
                
                for row in transform_matrices[i]:
                    updated_landmark.append(np.dot(landmarks_1[j],row))
           
                
                updated_landmarks.append(updated_landmark)
            updated_landmarks_all.append(updated_landmarks)
        
    return np.array(updated_landmarks_all, dtype=object)

In [161]:
directory_aligned = "Datasets/Original Foot Data/Bounding Box Aligned/"
directory_processed = "Datasets/Original Foot Data/Processed/"

In [214]:
sample_rate = 1 #read by skipping given number of lines

In [220]:
aligned_points_all = read_directory(directory_aligned, sample_rate)[0]
original_points_all, original_landmarks_all = read_directory(directory_processed, sample_rate)

In [221]:
trans_mats = calculate_transformation_matrix(original_points_all, aligned_points_all)

In [222]:
aligned_landmarks_all = calculate_aligned_landmarks(original_landmarks_all,trans_mats)

In [301]:
point_x = aligned_points_all[0][:,0]
point_z = aligned_points_all[0][:,2]

In [302]:
threshold = 0.5
point = (max(point_x) + min(point_x))/2

In [303]:
#np.where(np.logical_and(a>=6, a<=10))

In [345]:
def length_width_calculator(foot_data):
    
    threshold = 0.5
    
    lengths = []
    widths = []
    
    for foot in foot_data:
        point_x = foot[:,0]
        point_z = foot[:,2]

        length = 0
        for point in range(int(min(point_x)),int(max(point_x))):
            indices_filtered = np.where((point_x>=point-threshold)&(point_x<=point+threshold))
            point_z_filtered = point_z[indices_filtered]
            
            if(len(point_z_filtered)>0):
                length_point = max(point_z_filtered) - min(point_z_filtered)
            if length_point>length:
                length = length_point
                
        lengths.append(length)
                
        width = 0
        for point in range(int(min(point_z)),int(max(point_z))):
            indices_filtered = np.where((point_z>=point-threshold)&(point_z<=point+threshold))
            point_x_filtered = point_x[indices_filtered]
            
            if(len(point_x_filtered)>0):
                width_point = max(point_x_filtered) - min(point_x_filtered)
            if width_point>width:
                width = width_point
        
        widths.append(width)
    
    return lengths,widths

In [346]:
lengths,widths = length_width_calculator(aligned_points_all)

In [347]:
len(lengths)

53

In [348]:
len(widths)

53

In [349]:
lengths

[234.82001495000003,
 234.83002471999998,
 234.8299942,
 234.82999420000004,
 235.78000640999997,
 246.82000733,
 247.84996796,
 246.75998688000004,
 247.84000397,
 240.33300018,
 240.80001830999998,
 240.79002379999997,
 240.79000855,
 240.85000609999997,
 208.0840149,
 230.83001709,
 230.83001709,
 231.92002105,
 230.81000519,
 221.81000519,
 222.83002471999998,
 223.85002135999997,
 224.85001373,
 224.85999298000002,
 227.78002166000005,
 213.80003357,
 210.87998962000003,
 213.86997985999997,
 214.11199951000003,
 214.36799622,
 211.84001159,
 213.92500305,
 213.86999512000003,
 236.86000061,
 221.8099823,
 221.86997986,
 221.88002014,
 221.73997497,
 221.86000061,
 204.85002137,
 222.82000733,
 222.82003784,
 222.82000732999998,
 223.78001404000003,
 222.81997681000001,
 255.13599396,
 258.04650878999996,
 243.84003449,
 244.78999329000004,
 247.84001159999997,
 243.84002685999997,
 243.84005738000002,
 243.84005738000002]

In [350]:
widths

[90.52994919,
 90.32994842,
 90.61959839,
 90.62118530000001,
 90.71862793,
 98.67890548,
 98.67495728,
 95.39659882,
 98.45571136000001,
 88.8277855,
 88.99101257000001,
 89.01263809,
 89.02532196,
 88.92607498,
 74.35295105,
 82.67925262,
 82.50316620000001,
 82.57435989999999,
 82.63835144000001,
 85.17332459,
 84.9986763,
 84.94954681,
 84.7238617,
 84.66767121000001,
 84.27325439,
 83.48108292,
 80.71326065,
 83.51382065,
 83.48229979999999,
 83.63756561,
 80.90822220000001,
 83.63756562,
 82.37795257,
 95.80442048,
 83.35985565,
 83.54428863999999,
 83.66284561,
 83.67292785999999,
 83.71245575,
 77.42421341,
 89.84028626,
 89.84137726,
 89.88199616,
 89.78683853999999,
 89.87786865,
 93.60162735,
 93.6120224,
 92.45584107,
 92.53103638,
 93.93005371,
 92.52954482999999,
 92.60810852,
 92.62805176]

In [300]:
max(point_z)-min(point_z)

238.94398498000004